# 作業 : (Kaggle)鐵達尼生存預測

### 補充
* [pandas.DataFrame.nunique](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.nunique.html)

In [1]:
# 載入套件與資料
import pandas as pd
import numpy as np

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
df_train.shape

(891, 12)

In [2]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [4]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


# 作業1 
* 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )  
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因? 
  * 對 int/float datatype 並沒有問題
  * 只有 object datatype 的 mean() 與 max() 會有問題，原因是文字無法進行平均或比較大小。但我們可以發現，object 仍有部分欄位可傳回最大值，可以知道 object 欄位超過一定長度時會當成字串，此時會以字典順序做大小比較

# 作業2
* 思考一下，試著舉出今天三種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?
  * Ans: 應該是 object type 吧。
  * 舉例 : 複數 (a+bi)，因為需要兩個數字表示，雖然概念上是數值，但因為無法比較大小，因此只能歸類為"類別型欄位"。不論其他更奇怪的例子，都可以歸類為類別型欄位，因此類別型欄位應該最難處理

In [5]:
# object_features 的資料看起來怪怪的，再進一步看看
print("mean()/max()/unique() of int_features:")
print(df[int_features].mean())
print(df[int_features].max())
print(df[int_features].nunique())
print("\nmean()/max()/unique() of float_features:")
print(df[float_features].mean())
print(df[float_features].max())
print(df[float_features].nunique())
print("\nmean()/max()/unique() of object_features:")
print(df[object_features].mean())
print(df[object_features].max())
print(df[object_features].nunique())

mean()/max()/unique() of int_features:
Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64
Pclass    3
SibSp     8
Parch     9
dtype: int64
Pclass    3
SibSp     7
Parch     8
dtype: int64

mean()/max()/unique() of float_features:
Age     29.881138
Fare    33.295479
dtype: float64
Age      80.0000
Fare    512.3292
dtype: float64
Age      98
Fare    281
dtype: int64

mean()/max()/unique() of object_features:
Series([], dtype: float64)
Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object
Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64


In [6]:
# 會遇到 string 無法轉換成 float 的狀況，造成無法進行數學運算
test = 'Name'
# observe the values
print(df[test].head())
print(df[test].head().mean())

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
Name: Name, dtype: object


TypeError: Could not convert Braund, Mr. Owen HarrisCumings, Mrs. John Bradley (Florence Briggs Thayer)Heikkinen, Miss. LainaFutrelle, Mrs. Jacques Heath (Lily May Peel)Allen, Mr. William Henry to numeric